# Projeto 2 - Classificador Automático de Sentimento

Você foi contratado por uma empresa parar analisar como os clientes estão reagindo a um determinado produto no Twitter. A empresa deseja que você crie um programa que irá analisar as mensagens disponíveis e classificará como "relevante" ou "irrelevante". Com isso ela deseja que mensagens negativas, que denigrem o nome do produto, ou que mereçam destaque, disparem um foco de atenção da área de marketing.<br /><br />
Como aluno de Ciência dos Dados, você lembrou do Teorema de Bayes, mais especificamente do Classificador Naive-Bayes, que é largamente utilizado em filtros anti-spam de e-mails. O classificador permite calcular qual a probabilidade de uma mensagem ser relevante dadas as palavras em seu conteúdo.<br /><br />
Para realizar o MVP (*minimum viable product*) do projeto, você precisa implementar uma versão do classificador que "aprende" o que é relevante com uma base de treinamento e compara a performance dos resultados com uma base de testes.<br /><br />
Após validado, o seu protótipo poderá também capturar e classificar automaticamente as mensagens da plataforma.

## Informações do Projeto

Prazo: 19/Set até às 23:59.<br />
Grupo: 2 ou 3 pessoas - grupos com 3 pessoas terá uma rubrica diferenciada.<br /><br />
Entregáveis via GitHub: 
* Arquivo notebook com o código do classificador, seguindo as orientações abaixo.
* Arquivo Excel com as bases de treinamento e teste totalmente classificado.

**NÃO gravar a key do professor no arquivo**


### Entrega Intermediária: Check 1 - APS 2

Até o dia 10/Set às 23:59, xlsx deve estar no Github com as seguintes evidências: 

  * Produto escolhido.
  * Arquivo Excel contendo a base de treinamento e a base de testes já classificadas.

Sugestão de leitura:<br />
https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/

___

## Parte I - Adquirindo a Base de Dados

Acessar o notebook **Projeto-2-Planilha** para realizar a coleta dos dados. O grupo deve classificar os dados coletados manualmente.

___
## Parte II - Montando o Classificador Naive-Bayes

Com a base de treinamento montada, comece a desenvolver o classificador. Não se esqueça de implementar o Laplace Smoothing https://en.wikipedia.org/wiki/Laplace_smoothing

Opcionalmente: 
* Limpar as mensagens removendo os caracteres: enter, :, ", ', (, ), etc. Não remover emojis.<br />
* Corrigir separação de espaços entre palavras e/ou emojis.
* Propor outras limpezas/transformações que não afetem a qualidade da informação.

Escreva o seu código abaixo:

##  Empresa TESLA

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import math
from scipy import stats
import mpmath
import matplotlib.pyplot as plt
from emoji import UNICODE_EMOJI

In [2]:
#Leitura dos arquivos em Excel
TR1 = pd.read_excel('Treinamento1.xlsx')
TS1 = pd.read_excel('Teste1.xlsx')
TR2 = pd.read_excel('Treinamento2.xlsx')
TS2 = pd.read_excel('Teste2.xlsx')

In [3]:
def limpa(PD, coluna):
    
    tags =    ["@", "#", "http"]
    
    tabs =    ["\n", "\t"]
    
    pontos =  [":", "/", "+", "-", "&",
               "%"]
    
    deletar = [",", ".", "?", "@", ":",
               ";", "!", "'", '"', "“",
               "”", "#", "(", ")", "…",
               "/", "+", "-", "$", "&",
               "%", "*", "[", "]", "{",
               "}", "'s"]
    
    lista_um = []
    for frase in PD[coluna]:
        frase = frase.split()
        jumba = []
        for palavra in frase:
            jaca = palavra
            if ("tesla" in palavra) or ("tsla" in palavra):
                jaca = "tesla"
            elif "@" in palavra:
                jaca = ""
            elif "http" in palavra:
                jaca = ""
            elif "#" in palavra:
                jaca = ""
            elif (palavra == frase[-1]) and ("…" in palavra):
                jaca = ""
   
            jumba.append(jaca)
        lista_um.append(" ".join(jumba))
        
    lista_dois = []     
    for frase in lista_um:
        h = ""
        for k in frase:
            a = k
            if k in UNICODE_EMOJI:
                a = " "+ k + " "
            elif k in pontos:
                a = " "+ k + " "
            elif k in tabs:
                a = " "
            h += a
        lista_dois.append(h)
    
    lista_tres = []
    for frase in lista_dois:
        for n in deletar:
            frase = frase.replace(n , "")
        lista_tres.append(frase)
        
    lista_quatro = []
    for frase in lista_tres:
        frase = frase.split()
        gogo = []
        for palavra in frase:
            if palavra in UNICODE_EMOJI:
                palavra = palavra
            elif len(palavra) <= 2:
                palavra = ""
            elif palavra == "rt":
                palavra = ""
            gogo.append(palavra)
        lista_quatro.append(" ".join(gogo))
        
    lista_cinco = []
    z = 1
    for frase in lista_quatro:
        nano = " ".join(frase.split())
        if nano == "":
            nano = "tesla"
        lista_cinco.append(nano)
    
    PD[coluna]  = pd.Series((lista_cinco), index = PD.index)

In [4]:
def vai(PD_treinamento, PD_teste, classificacaoes):
    
    supimpa = {"Geral":{}, "Chance": {}}
    limpa(PD_treinamento, "Tweets")
    limpa(PD_teste, "Tweets")
    
    for frase in PD_treinamento["Tweets"]:
        frase = frase.split()
        for palavra in frase:
            if palavra in supimpa["Geral"]:
                supimpa["Geral"][palavra] += 1
            else:
                supimpa["Geral"][palavra] = 1
                
    for classi in classificacaoes:
        TTT = PD_treinamento[(PD_treinamento['Classificação']==classi)]
        
        supimpa["Chance"][classi] = len(TTT['Classificação']) / len(PD_treinamento['Classificação'])
        
        supimpa[classi] = {}
        for frase in TTT["Tweets"]:
            frase = frase.split()
            for palavra in frase:
                if palavra in supimpa[classi]:
                    supimpa[classi][palavra] += 1
                else:
                    supimpa[classi][palavra] = 1  
                    
    numero_divisor = len(supimpa["Geral"])
    soma_geral = sum(supimpa["Geral"].values())
    for palavra in supimpa["Geral"]:
        supimpa["Geral"][palavra] = (supimpa["Geral"][palavra] + 1) / (soma_geral + numero_divisor)
        
    for classi in classificacaoes:
        soma_classi = sum(supimpa[classi].values())
        for palavra in supimpa[classi]:
            supimpa[classi][palavra] = (supimpa[classi][palavra] + 1) / (soma_classi + numero_divisor)
        supimpa[classi]["palavra_nao_existe"] = (1) / (soma_classi + numero_divisor)
    
    chances = {}
    u = 1 
    for frase in PD_teste["Tweets"]:
        chances[frase] = {}
        frasum = frase.split()
        for classi in classificacaoes:
            probabilidade = mpmath.mpf(1.0)
            for palavra in frasum:
                if palavra in supimpa[classi]:
                    probabilidade *= supimpa[classi][palavra]
                else:
                    probabilidade *= supimpa[classi]["palavra_nao_existe"]
                    
            chances[frase][classi] = probabilidade * supimpa["Chance"][classi]
    
    lista_legal = []
    for frase in PD_teste["Tweets"]:
        sugar = 0
        valor = 0
        for classi in classificacaoes:
            if chances[frase][classi] > valor:
                valor = chances[frase][classi]
                sugar = classi
        lista_legal.append(sugar) 
        
    PD_teste["Naive-Bayes"]  = pd.Series((lista_legal), index = PD_teste.index) 
    
    lista_joia = []
    for op in PD_teste['Classificação']:
        lista_joia.append(op)
        
    
    lista_uber = []
    for d in range(len(lista_legal)):
        if (lista_legal[d] == lista_joia[d]):
            lista_uber.append("Verdadeiro")
        else:
            lista_uber.append("Falso")
            
    PD_teste["Final"]  = pd.Series((lista_uber), index = PD_teste.index)  
    
    verdadeiro = 0
    falso = 0
    for i in range(len(lista_uber)):
        if lista_uber[i] == "Verdadeiro":
            verdadeiro += 1
        else:
            falso += 1
    
    duper = {}
    for classi in classificacaoes:
        duper[classi] = {}
        for op in classificacaoes:
            duper[classi][op] = 0
            
    
    
    for i in range(len(PD_teste['Classificação'])):
        duper[PD_teste.loc[i, 'Classificação']][PD_teste.loc[i, "Naive-Bayes"]] += 1
        
        
    for op in duper:
        total = sum(duper[op].values())
        for classi in duper[op]:
            duper[op][classi] = duper[op][classi] / total

    
    taxa_de_acerto = (verdadeiro / len(lista_uber))
    
    return taxa_de_acerto, duper

In [5]:
def mostra(PD_treinamento, PD_teste, classificacaoes):
    taxa, dicti = vai(PD_treinamento, PD_teste, classificacaoes)
    print("Com {} categorias, esse classificador teve uma taxa de acerto geral de {:.2f}%".format(, taxa * 100))
    print("As taxas individuais seguem:")
    print("\t")
    for op in dicti:
        for classi in dicti[op]:
            print("{:05.2f}% dos tweets {}s foram classificados como {}".format(dicti[op][classi] * 100, op, classi))
        print("\t")
    

___
## Verificando a performance

Agora você deve testar o seu Classificador com a base de Testes.<br /><br /> 

Você deve extrair as seguintes medidas:
* Porcentagem de positivos falsos (marcados como relevante mas não são relevantes)
* Porcentagem de positivos verdadeiros (marcado como relevante e são relevantes)
* Porcentagem de negativos verdadeiros (marcado como não relevante e não são relevantes)
* Porcentagem de negativos falsos (marcado como não relevante e são relevantes)

Obrigatório para grupos de 3 alunos:
* Criar categorias intermediárias de relevância baseado na diferença de probabilidades. Exemplo: muito relevante, relevante, neutro, irrelevante e muito irrelevante.

In [6]:
mostra(TR1, TS1, ["Irrelevante", "Relevante"])

Esse classificador teve uma taxa de acerto geral de 84.00%
As taxas individuais seguem:
	
79.17% dos tweets Irrelevantes foram classificados como Irrelevante
20.83% dos tweets Irrelevantes foram classificados como Relevante
	
14.47% dos tweets Relevantes foram classificados como Irrelevante
85.53% dos tweets Relevantes foram classificados como Relevante
	


In [7]:
mostra(TR2, TS2, ["Irrelevante", "Negativo", "Neutro", "Positivo"])

Esse classificador teve uma taxa de acerto geral de 63.50%
As taxas individuais seguem:
	
45.83% dos tweets Irrelevantes foram classificados como Irrelevante
04.17% dos tweets Irrelevantes foram classificados como Negativo
29.17% dos tweets Irrelevantes foram classificados como Neutro
20.83% dos tweets Irrelevantes foram classificados como Positivo
	
03.23% dos tweets Negativos foram classificados como Irrelevante
79.03% dos tweets Negativos foram classificados como Negativo
14.52% dos tweets Negativos foram classificados como Neutro
03.23% dos tweets Negativos foram classificados como Positivo
	
06.25% dos tweets Neutros foram classificados como Irrelevante
09.38% dos tweets Neutros foram classificados como Negativo
64.06% dos tweets Neutros foram classificados como Neutro
20.31% dos tweets Neutros foram classificados como Positivo
	
03.85% dos tweets Positivos foram classificados como Irrelevante
07.69% dos tweets Positivos foram classificados como Negativo
30.77% dos tweets Positivo

___
## Concluindo


Com o código pronto e funcionando, concluiu-se que a empresa Tesla possui uma maior quantidade de mensagens Relevantes Verdadeiro (84%) do que Falsos (16%), o que demonstra que o código feito foi eficiente, pois conseguiu, dentre as opções de calssificação existentes, classificar os tweets da maneira adequada. E, tendo em vista o objetivo da empresa - melhorar seu marketing a fim de ampliar a divulgação e o consumo de seus produtos -, foi preferível obter todas as mensagens relevantes mesmo que com elas tenham vindo algumas não relevantes porque a porcentagem dessas dentre as demais foi pequena, possibilitando que esses dados sejam removidos manualmente.

A empresa deveria seguir financiando o projeto, pois ele retorna o que o público alvo pensa sobre seus produtos e, com isso, saber o que deve ser aprimorado. E com mais verba seria possível aumentar o data base de tweets classificados, aumentando, assim, a probabilidade de acertos.

Além de o projeto ajudar a Tesla, ele nos abre porta para as demais empresas, pois como nossa programação está escrito com funções, isso permite que analisamos não só os dados desse contexto quanto os demais.  



